<a href="https://colab.research.google.com/github/Jeeva55t/Automatic_email_reply_AGENT_LLM/blob/main/Automatic_reply_email_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install -q langchain langchain_community google-genai

In [63]:
from langchain.agents import initialize_agent,Tool
from langchain.agents import AgentType
from langchain.tools import tool
import imaplib,smtplib,email
from email.mime.text import MIMEText
import os

In [64]:
GEMINI_API_KEY = "UPLOAD_YOUR_KEY"
EMAIL = "jeevanandk95@gmail.com"
PASSWORD = "eqip hvkb myrv gtau"

In [65]:
@tool
def read_gmail(query: str = "") -> str:
  """ Reads the latest unread Gmail message."""
  mail = imaplib.IMAP4_SSL("imap.gmail.com")
  mail.login(EMAIL,PASSWORD)
  mail.select("inbox")
  status , data = mail.search(None,"UNSEEN")
  ids = data[0].split()
  if not ids:
    return "No unread emails found"

  latest_id = ids[-1]
  status,msg_data = mail.fetch(latest_id,"(RFC822)")
  msg = email.message_from_bytes(msg_data[0][1])
  subject = msg["Subject"]
  sender = msg["From"]
  body = ""

  if msg.is_multipart():
    for part in msg.walk():
      if part.get_content_type() == "text/plain":
        body += part.get_payload(decode=True).decode()
  else:
    body += msg.get_payload(decode=True).decode()

  return f"Subject: {subject}\nFrom: {sender}\nBody: {body}"

In [66]:
#Tool - send email
@tool
def send_gmail_reply(details : str) -> str:
  """"Sends a reply to an email. Format: 'to | subject | message '. """
  try :
    to_email,subject,body = details.split("|",2)
    msg =  MIMEText(body)
    msg["Subject"] = "Re :" + subject
    msg["From"] = EMAIL
    msg["To"] = to_email
    server = smtplib.SMTP_SSL("smtp.gmail.com",465)
    server.login(EMAIL,PASSWORD)
    server.send_message(msg)
    server.quit()
    return "Email sent successfully"
  except Exception as e:
    return f"Error sending email: {str(e)}"

In [67]:
# from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    temperature = 0.3,
    api_key = GEMINI_API_KEY
)

In [68]:
tools = [read_gmail,send_gmail_reply]

In [69]:
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

In [70]:
task = """Read the latest unread email.If it is a query,write a polite reply and send it back.To send reply ,format must be : to|subject|message
"""
agent.run(task)



> Entering new AgentExecutor chain...
```json
{
  "action": "read_gmail",
  "action_input": ""
}
```
Observation: Subject: Fwd: Complaint on the Product
From: Jeevanand V <jeevanandv63@gmail.com>
Body: ---------- Forwarded message ---------
From: Jeevanand V <jeevanandv63@gmail.com>
Date: Mon, 14 Jul 2025 at 10:22
Subject: Complaint on the Product
To: <jeevanandk95@gmail.com>


I have ordered a headset from your website. The delivered headset is not
working properly.Could you kindly replace a new product.

Thought:```json
{
  "action": "send_gmail_reply",
  "action_input": "jeevanandv63@gmail.com|Re: Complaint on the Product|Dear Jeevanand V,\n\nThank you for contacting us regarding the issue with your headset. We sincerely apologize for the inconvenience this has caused.\n\nWe would be happy to replace your faulty headset with a new one. To process your replacement, could you please provide us with your order number and a photo of the defective headset?\n\nOnce we receive this infor

'Email sent successfully.'